# LinUCB 

## Principle

+ Learn LinUCB with or without SLA protect (reactive) option,
+ The aim is to avoid Latency mean metric in learning and test context of LinUCB
+ Test LinUCB without SLA protect when Latency is NOT in context, use SLA protect if Latency is in context
+ HPA and Basic Baseline tests can or not use SLA protect and Latency metric in context. 
+ Buffer Fillg Rate (aka BFR) policy, is another baseline. It don't need to know the Latency. Up to you to activate or not SLA protect mode in Bas
+ Basic policy particularities:
  +  needs the latency and acceptable gap information
  +  oscillates around the tipping point when the slope of the latency is stiff
+  HPA policy particularities:
   +  needs latency information
+ Training is performed with Episodes of 1 step. At each episode a workload is choosen and a state (with a buffer value) is choosen randomly. The ON_TERMINATE option of the environmenent does not make sense here.

## Results

+ La normalisation (minmax=[0,1]) de l'espace d'observation donnent de mauvais résultats dés l'entrainement
+ Si on définit une ou des actions négatives, l'entrainement sans le mod REACT fonctionne mieux, car sinon on n'apprend pas bien sur le bras UP
+ Autant les erreurs de réglage dans la plage OVER s'avère acceptables, il faut absolument trouver un moyen d'éviter les VIOLATION
+ Résolu? 
  Si ALPHA est trop petit (0.3 et moins), on a une erreur: 
```  
  chosen_arm = np.random.choice(candidate_arms)
     93 return chosen_arm
  File mtrand.pyx:934, in numpy.random.mtrand.RandomState.choice()
```  

### Labo mode

+ REACT à True (Training and Test)
  + LinUCB Train CReg: **359**
  + KFOOFW LinUCB CReg:**10.64**  => C'est très bon!
  + BFR CReg:1758.59
  + Hpa CReg:546.45
  + Basic CReg: **116.43**

+ REACT à False (Training) and True (Test). 
  + LinUCB Train CReg: **493**
  + KFOOFW LinUCB CReg: **135.12**
  + BFR CReg:1758.59
  + Hpa CReg:546.45
  + Basic CReg:116.43
  + Les plus mauvais scores (forcément en descendant!!):
    + E28  30 1.0 4 Un 0.98 20.0ms DBSize:7052MB Dft DBA:5632.0MB  CREG: 35
      + En mode descendant. On s'arrête trop top, assez loin de la cible, pourquoi ? => regarder les données
    + E90  22 1.0 12 Pa 0.98 20.0ms DBSize:5233MB Dft DBA:4096.0MB CREG: 7
      + En mode descendant. On s'arrete trop top, pourquoi ? => regarder les données

### Prod mode

+ REACT à False (Training and Test)
  + LinUCB Train CReg: **606** (then 584)
  + KFOOFW LinUCB CReg: **223.79** (then 199)
  + BFR CReg: 2316.84
  + Les plus mauvais scores:
    + ['E14 S63 8192.0MB', 34.7158, **1.2138**, 0], ['E15 S63 128.0MB', 4.714, **35.726**, 0] E15  15 1.0 6 Sp 0.98 20.0ms DBSize:3540MB Dft DBA:2816.0MB
      + E14 On descend un peu trop mais on est bien
      + E15 On cherche à descendre encore mais on est bien
    + ['E19 S63 128.0MB', 5.8779, 30.0268, 0] E19  22 1.0 3 Sp 0.98 20.0ms DBSize:5167MB Dft DBA:4096.0MB
      + On devrait remonter un peu mais rien de grave (SLA_OK)
    + [['E92 S63 8192.0MB', 25.2901, 9.1486, 0], ['E93 S63 128.0MB', 3.0087, 7.7973, 0]] E92  40 0.3 8 Un 0.98 20.0ms DBSize:2867MB Dft DBA:2176.0MB
      + On reste  à OVER en descente, donc c'est un "mauvais" réglage
      + A la remontée, on reste à **VIOLATION** => on est très mauvais
    + Idem E91  22 1.0 12 Pa 0.98 20.0ms DBSize:5233MB Dft DBA:4096.0MB, **VIOLATION**
    + ['E94 S63 8192.0MB', 3.8052, 22.1507, 0], ['E95 S63 128.0MB', -37.5559, 75.1117, 0] E95  40 1.0 8 Pa 0.98 20.0ms DBSize:9441MB Dft DBA:7552.0MB
      + BFR et HPA-Buf sont très mauvais à BUF=128MB. Il s'agit surement d'un pb de mesure prise pendant que le buffer était réorganisé. Le filling rate trop faible (89%), l'ago décide de rester à ce niveau au lieu de remonter
      + Au final, même lorsque la mesure est prise dans de mauvaise condition, le contexte plus large de LinUCB, permet de mieux fonctionner
  + Globalement, on peut dire que les résultats ne sont pas mauvais, mais juste moins précis (pb de la zone OK) que en mode LABO

+ REACT à True (Training) and False (Test). 
  + LinUCB Train CReg: **409**
  + KFOOFW LinUCB CReg: **1442.19**
  + BFR CReg:2316.84
  + Les plus mauvais scores: Beaucoup, car on n'a pas appris à remonter
  + **Ce UC n'a aucun sens**


+ Conclusion:
  + Il faut un indicateur de "réglage satisfaisant" car on accumule du regret parfois dans des conditions pourtant OK
  + Le mode REACT améliore l'apprentissage mais il n'est pas pertinent si on ne peut pas l'utiliser en TEST


## Notes HSLinUCB

+ Donne de moins bons résultats pour l'instant que KFOO

## Notes KFOOLinUCB

+ Un ALPHA=0.5 est pas mal, COVERAGE=2, FullSequential DSSelector
+ On a observé un modèle ayant obtenu un meilleur (de peu) résultat d'apprentissage qui performe moins bien en test
+ Questionnement sur Rewards V3 vs V3E...
+ Le training en mode aléatoire (Gaussien) ne se différentie pas beaucoup du mode séquentiel
+ Fort impact de TRAINING_USE_SLA_PROTECT à True (meilleure). On considère qu'on peut utiliser la latence pendant l'entrainement pour le calcul de Reward. 
+ Ne pas activer TEST_USE_SLA_PROTECT, si on n'a pas la latence du client après déploiement en Prod.
+ Usage ou pas de la latence dans l'observation. Attention, si on l'utilise, on devra aussi l'avoir après déploiement de l'agent en Production!!!
+ A noter que parfois des zones "OK" sont coincées entre 2 zones "OVER". Le regret cumulé en est impacté. 
+ Qq constat dans la config de learning avec ds_train3, TRAINING_SLA_PROTECT=True et TEST_SLA_PROTECT=False, Reward=ADBMSBufferCacheRewardContinous. 
  + Si 10, 12 clients sont dans le dataset, la performance du modèle se dégrade pour ces 2 cas
  + En Test:
    + On voit que on a  un regret qui suit le regret HPA. C'est normal vu notre calcul de Reward (Continous @Pat)?
    + Si on fait des tests de descente de buffer uniquement, les résultats sont bien meilleurs que les tests avec descente + montée. Les tests de montée seuls suivent majoritairement le regret HPA ou font mieux.
  + Essais aux différentes valeurs de perf_level (default=0.98, Toutes: [0.9   0.95  0.965 0.98  0.99  0.995 0.997]):
    + A perf maximale, c'est la catastrophe, ça ne marche pas, l'apprentissage a échoué
    + C'est plus simple d'apprendre à niveau de perf plus faible

In [ ]:
import sys
import os
sys.path.append(os.path.abspath(os.pardir))

import bandits.datasource.adbms_dataframe as ds
import bandits.reward as rw
from bandits.gym_env import VSEnv, VSMonitor
from bandits.agent import VSAgent

# Parameters
RND_SEED = 4242
#qpslat_weights="01"   # QPS is not used in performance computation (weigth is 0%), the objective is to maintain te Latency (weigth is 100%). Other options is "19" (10% for QPS, 90% for Latency)

PLOTLY_RENDERER="svg" # "plotly_mimetype+notebook": Ex: png, jpg, svg
IPERF_LEVEL=0.98    # [0.9   0.95  0.965 0.98  0.99  0.995 0.997]
VERSION=9

# Choose ratio=80 for 80% train and 20% test. Choose 0 for a separation by clients. Ex Train on all Odd wl_clients num and test on even wl_clients bum
datasets = ds.TrainTestDataSets(testastrain=False, ratio=0, version=VERSION, perf_level=IPERF_LEVEL, randtypes=[ "special", "gaussian", "uniform", "pareto" ], clients=[1,2,3,4,5,6,7,8,9,10,11,12], seed=42, verbose=1)    

In [ ]:

OBSERVATION_SPACE_ELEMS = [
####    "tables",                      # Not significant while in production the DB schema is totaly different than the one built by Sysbench. DB Size should be sufficient
####    "tables_rows",                 # Not significant while in production the DB schema is totaly different than the one built by Sysbench. DB Size should be sufficient
#    "iperf01",                      # Current performance indicator based on WPS and Latency, 01 is computed with weigths as follow: 0% QPS and 100% Latency
    "db_size_mb",                    # Initial DB size after creation of all tables+indexes and before the workload with all clients
    "sysbench_filtered.latency_mean",      # [0, 1000] ms. Filtered latency from client. In inference/live mode, it should be a predicted value (see predictor.ipynb using XGBoost)
#    "buf_size_idx",                      # Index form of the buffer size value [0, 63]
    "observation.normalized_buf_size",     # [0., 1.] Taille normalisée courante du buffer cache
#    "perf_target_level",           # [0., 1.] Objectif de performance cible. Est une valeur fixe lors des entrainements (1 modèle pour 1 objectif de perf)
#    "latency_threshold",           # In ms. Est une valeur fixe lors des entrainements
    "observation.cache_used_pages_ratio",
    "observation.cache_hit_ratio",
    "extra_info.usage.db_size_initial.IndexSizeMB", # Initial Indexes DB size 
    "extra_info.usage.qps",          # QPS picked on server ("Questions" Global status variable throughput)
    "extra_info.global_status.KBps", # Kilo Bytes per seconds picked on server with "Bytes_sent" Global status variable throughput
    "observation.write_wait_ratio",  # Seems not relevant and usable on MariaDB
#    "extra_info.global_status.threads_running_start", # NOT POWERFULL!!?? The number of connections currently running an SQL query. Be carefull of a Bug in some MariaDB versions!! => TOO MUCH NOISE ON ITS VALUE!!!!!
    "extra_info.global_status.created_tmp_tables_diff", # ?
    "extra_info.global_status.handler_delete_diff", # ?
    "extra_info.global_status.handler_update_diff", # ?
    "extra_info.global_status.handler_write_diff", # ?
    "extra_info.global_status.innodb_buffer_pool_read_requests_diff",
    "extra_info.global_status.innodb_buffer_pool_write_requests_diff",
    "extra_info.global_status.innodb_buffer_pool_wait_free_diff",
    "extra_info.global_status.innodb_data_reads_diff",
    "extra_info.global_status.innodb_data_written_diff",
    "extra_info.global_status.max_used_connections_end",
    "extra_info.global_status.threads_running_end",
    "extra_info.global_status.threads_connected_end",
    "extra_info.global_status.innodb_buffer_pool_bytes_dirty_end", # Seems cool to have it
    "extra_info.global_status.innodb_buffer_pool_pages_free_end",  # Seems cool to have it
#    "extra_info.global_status.innodb_buffer_pool_pages_data_diff",
    "extra_info.global_status.innodb_buffer_pool_bytes_dirty_diff", # Seems cool to have it
    "extra_info.global_status.innodb_buffer_pool_pages_free_diff",  # Seems cool to have it
    "extra_info.global_status.information_schema.innodb_metrics.lock_row_lock_time_max",
    "extra_info.global_status.information_schema.innodb_metrics.lock_row_lock_waits",
    "extra_info.global_status.information_schema.innodb_metrics.os_pending_reads",
    "extra_info.global_status.information_schema.innodb_metrics.os_pending_writes",
    "extra_info.global_status.information_schema.innodb_metrics.file_num_open_files",
    "extra_info.global_status.information_schema.innodb_buffer_pool_stats.MODIFIED_DATABASE_PAGES", # Sounds COOL to have it!!
    "extra_info.global_status.information_schema.innodb_buffer_pool_stats.PAGES_READ_RATE",
    "extra_info.global_status.information_schema.innodb_buffer_pool_stats.PAGES_CREATE_RATE",
#    "extra_info.global_status.information_schema.innodb_buffer_pool_stats.PAGES_WRITTEN_RATE",  # CATASTROPHIC at an incredible point !!!
    "extra_info.global_status.information_schema.innodb_buffer_pool_stats.READ_AHEAD_RATE",
    "extra_info.global_status.information_schema.innodb_buffer_pool_stats.READ_AHEAD_EVICTED_RATE",
    "extra_info.global_status.information_schema.innodb_buffer_pool_stats.PAGES_MADE_YOUNG_RATE", # Sounds COOL to have it!!
    "extra_info.global_status.information_schema.innodb_buffer_pool_stats.POOL_SIZE", # Sounds COOL to have it!!
    "extra_info.global_status.information_schema.innodb_buffer_pool_stats.FREE_BUFFERS",
    "extra_info.global_status.information_schema.innodb_buffer_pool_stats.DATABASE_PAGES",
    "extra_info.global_status.information_schema.innodb_buffer_pool_stats.OLD_DATABASE_PAGES",
    "extra_info.global_status.information_schema.innodb_buffer_pool_stats.PENDING_DECOMPRESS",       #?
    "extra_info.global_status.information_schema.innodb_buffer_pool_stats.PENDING_READS",       #?
    "extra_info.global_status.information_schema.innodb_buffer_pool_stats.PENDING_FLUSH_LRU",       #?
    "extra_info.global_status.information_schema.innodb_buffer_pool_stats.PENDING_FLUSH_LIST",       #?
    "extra_info.global_status.information_schema.innodb_buffer_pool_stats.PAGES_MADE_YOUNG",
    "extra_info.global_status.information_schema.innodb_buffer_pool_stats.PAGES_NOT_MADE_YOUNG",
    "extra_info.global_status.information_schema.innodb_buffer_pool_stats.PAGES_MADE_NOT_YOUNG_RATE",
    "extra_info.global_status.information_schema.innodb_buffer_pool_stats.NUMBER_PAGES_READ",       #?
    "extra_info.global_status.information_schema.innodb_buffer_pool_stats.NUMBER_PAGES_CREATED",    #?
    "extra_info.global_status.information_schema.innodb_buffer_pool_stats.NUMBER_PAGES_WRITTEN",    # ? 
    "extra_info.global_status.information_schema.innodb_buffer_pool_stats.NUMBER_PAGES_GET",
    "extra_info.global_status.information_schema.innodb_buffer_pool_stats.HIT_RATE",
    "extra_info.global_status.information_schema.innodb_buffer_pool_stats.YOUNG_MAKE_PER_THOUSAND_GETS",         #?
    "extra_info.global_status.information_schema.innodb_buffer_pool_stats.NOT_YOUNG_MAKE_PER_THOUSAND_GETS",         #?
    "extra_info.global_status.information_schema.innodb_buffer_pool_stats.NUMBER_PAGES_READ_AHEAD",         #?
    "extra_info.global_status.information_schema.innodb_buffer_pool_stats.NUMBER_READ_AHEAD_EVICTED",         #?
    "extra_info.global_status.information_schema.innodb_buffer_pool_stats.LRU_IO_TOTAL",
    "extra_info.global_status.information_schema.innodb_buffer_pool_stats.LRU_IO_CURRENT",
#    "extra_info.global_status.information_schema.innodb_buffer_pool_stats.UNCOMPRESS_TOTAL",
#    "extra_info.global_status.information_schema.innodb_buffer_pool_stats.UNCOMPRESS_CURRENT",
]

ACTION_MINMAX = (-1,1)  # The Min and Max action values. Arms count and the position of the "STAY" arm are depending on it.
ON_TERMINATE=-1 # Environement option. -1=Never teminate (the default, just truncate on Timelimit), 0=Terminate with Zero bonus on reward, N=Terminate with a bonus added to the reward (may exceed 1)
TRAINING_COVERAGE=1.2 # Coverage of the total states present in dataset. 1.0 means that almost all the dataset will be covered (depends also on the random distribution that may repeat twice the same state)
TRAINING_STEPS_PER_EPISODE = 64    # Important to take benefit of the decreasing exploration vs explotation tradeoff
TRAINING_VERBOSITY=5
NORMALIZE_OBS=False # Sounds a bad idea to normalize with LinUCB


# Explore vs exploit
ALPHA=0.99 #0.5 

# In Labo mode, we consider that having usage of client's latency in LinUCB context, in both learning and test, is not an issue
# In Prod mode, we can access client's latency only during learning process. Latency cannot be part of the LinUCB context. The only viable and comparable baseline is BFR (Buffer Filling Rate) baseline.
LABO_EXPE=False

# Labo or Prod ?
if LABO_EXPE:
    TRAINING_USE_SLA_PROTECT=True #use or NOT Use the reactive mode to protect the SLA. Let see where we are going without this protection...
    TEST_USE_SLA_PROTECT = True #True if "sysbench_filtered.latency_mean" in OBSERVATION_SPACE_ELEMS else False
else:
    TRAINING_USE_SLA_PROTECT=False #It makes no sense to put it at True if we cannot use it in test..
    TEST_USE_SLA_PROTECT = False   # Must be like that (False)
    OBSERVATION_SPACE_ELEMS.remove("sysbench_filtered.latency_mean") # Ensure latency is not used in context

# Discrete reward is either 0 or 1 depending on the Arm (is it the good one or not?). 
# Continous reward in range [-1., 1.] is computed. The best arm is the one having the maximum value
reward_learning = rw.ADBMSBufferCacheRewardContinousV3E(action_minmax=ACTION_MINMAX)
reward_test = rw.ADBMSBufferCacheRewardContinousV3E(action_minmax=(-1,1))
reward_hpa = rw.ADBMSBufferCacheRewardContinousV3E(action_minmax=(-63,63)) 


In [ ]:
SIGMA_TIPPING_RATIO=-1. #-1 #0.9375 # The sigma value for the random gaussian distribution around the "mean" (or "mu"), the point of SLA tipping. If <0 use the auto-computing based on tipping point.
# 1- UCB confidence is <0 !!??
#ds_train = ds.ADBMSBufferCacheStatesFullRandomSelector(datasets.df_train, sigma_tipping_ratio=SIGMA_TIPPING_RATIO, seed=RND_SEED)
# 2- ok but longer than 3
ds_train = ds.ADBMSBufferCacheStatesRandomBufferSelector(datasets.df_train, sigma_tipping_ratio=SIGMA_TIPPING_RATIO, seed=RND_SEED)
# 3- sligthly better than ds_train5
#ds_train = ds.ADBMSBufferCacheStatesSequentialSelector(datasets.df_train, topdown=True) # At each reset choose the workload from the current episode index, go to the top or bottom state of the workload as specified by topdown argument.
# 4- Optimal ?
# Choix PR
#ds_train = ds.ADBMSBufferCacheStatesFullSequentialSelector(datasets.df_train)  # Start at the begining of the dataset and go to the next state at each reset. When bottom state is reached, next reset will go to the next workload (auto rewind) 
# 5-
#ds_train = ds.ADBMSBufferCacheStatesTopDownRockerSelector(datasets.df_train)            # Start sequentialy each workload twice. First from Top, second from Bottom

ds_train.prepareContext(OBSERVATION_SPACE_ELEMS, normalize=NORMALIZE_OBS)

K_ARMS = reward_learning.actions.count()
K_ARM_STAY = reward_learning.actions.armStay()          # The index of the "STAY" arm
print(f'K_ARMS:{K_ARMS} STAY_ARM_IDX:{K_ARM_STAY}')


In [ ]:
from policy_linucb_kfoofw import LinUCBPolicy_kfoofw
from policy_linucb_hs import HSLinUCBPolicy

# Creates an environement that will change the workload at each Episode and perform only 1 step per episode (better for Exploration vs Exploitation random distribution)
env_train = VSMonitor(VSEnv(ds_train, reward=reward_learning, notify_react=TRAINING_USE_SLA_PROTECT, max_steps_per_episode=TRAINING_STEPS_PER_EPISODE, on_terminate=ON_TERMINATE, verbose=0))

#policy = LinUCBPolicy_kfoofw(reward_learning.actions, ctx=ds_train.contextElems(), alpha=ALPHA)
policy = HSLinUCBPolicy(reward_learning.actions, ctx=ds_train.contextElems(), alpha=ALPHA)

linucb_agent = VSAgent(policy)
linucb_agent.learn(env_train, TRAINING_COVERAGE, verbose=TRAINING_VERBOSITY)
env_train.close()


In [ ]:
#linucb_agent.showLearnFig(head_title="LinUCB-KFOOFW", renderer=PLOTLY_RENDERER)
linucb_agent.showLearnFig(head_title=f"LEARN/{policy.name} TotalSteps:{env_train.unwrapped.total_steps}", renderer=None)
linucb_agent.save(filever=str(VERSION), verbose=1, min_max_scaler=ds_train.min_max_scaler)

## Tests

The DataSet Selector for Tests is a sewuential one. It parses workloads and buffer sizes sequentially.

In [ ]:
#ds_test = ds.ADBMSBufferCacheStatesSequentialSelector(datasets.df_test, topdown=True)  # Start sequentialy each Workload at Top or from Bottom depending on "topdown" argument
ds_test = ds.ADBMSBufferCacheStatesTopDownRockerSelector(datasets.df_test)               # Start sequentialy each workload twice. First from Top, second from Bottom

TEST_EPISODES_COUNT = -2    # If <0, abs(EPISODES_COUNT) is a multiple of workloads count 
TEST_STEPS_PER_EPISODE = -1 # If <0, abs(STEPS_PER_EPISODE) is a multiple of buffer values count 


### LinUCB Test

With learned model.

In [ ]:
#linucb_agent = VSAgent(LinUCBPolicy_kfoofw(actions=ACTION_MINMAX))
#files_list, optdict = linucb_agent.load(filever=str(VERSION), verbose=1)
#ds_test.prepareContext(OBSERVATION_SPACE_ELEMS, normalize=NORMALIZE_OBS, with_scaler=optdict["min_max_scaler"]) # "with_scaler" is ignored if No normalization is enabled
ds_test.prepareContext(OBSERVATION_SPACE_ELEMS, normalize=NORMALIZE_OBS, with_scaler=ds_train.min_max_scaler) # "with_scaler" is ignored if No normalization is enabled

# Environement for test with a Reward (coming from the learning sessions). 
env_test = VSMonitor(VSEnv(ds_test, reward=reward_learning, notify_react=TEST_USE_SLA_PROTECT, max_steps_per_episode=TEST_STEPS_PER_EPISODE, verbose=1))

linucb_agent.predict(env_test, episodes_max=TEST_EPISODES_COUNT)


graph_tests = env_test.graphPerWorkload(f"Cumulated Regret/workload. Trained on: {env_train.unwrapped.msgStatus()} {env_train.unwrapped.desc()}")
graph_tests.addCurve(f"{linucb_agent.policy.name} CReg:{round(env_test.getRegretPerformance(),2)}", y=env_test.resultPerWorkload(), perf=env_test.getRegretPerformance())

env_test.close()

In [ ]:
#linucb_agent.showPredictFig(head_title="KFOOFW LinUCB", renderer=PLOTLY_RENDERER)
#linucb_agent.showPredictFig(head_title=f"TEST/{policy.name}")


### Baseline test with HPA (threshold detection)

In [ ]:
if LABO_EXPE:
    HPA_OBSERVATION_SPACE_ELEMS = [
        "buf_size_idx",                      # Index form of the buffer size value [0, 63]
        "sysbench_filtered.latency_mean",    # [0, 1000] ms. Filtered latency from client. In inference/live mode, it should be a predicted value (see predictor.ipynb using XGBoost)
        "latency_threshold",                 # Latency threshold for the SLA
    ]
    ext_hpa_threshold = None
    hpatyp = "Lat"
else:
    HPA_OBSERVATION_SPACE_ELEMS = [
        "buf_size_idx",                      # Index form of the buffer size value [0, 63]
        "observation.cache_used_pages_ratio",    # Buffer cache fill rate in [0., 1.]
    ]
    ext_hpa_threshold = 0.99
    hpatyp = f"Buf{ext_hpa_threshold}"

from policy_baselines import HpaPolicy

In [ ]:
ds_test.prepareContext(HPA_OBSERVATION_SPACE_ELEMS)

env_test = VSMonitor(VSEnv(ds_test, reward=reward_hpa, notify_react=TEST_USE_SLA_PROTECT, max_steps_per_episode=TEST_STEPS_PER_EPISODE, verbose=1))

policy = HpaPolicy(reward_hpa.actions, ctx=ds_test.contextElems(), ext_threshold=ext_hpa_threshold, debug=False)
hpa_agent = VSAgent(policy)
# No traning with this policy
# Test it!
hpa_agent.predict(env_test, episodes_max=TEST_EPISODES_COUNT)

graph_tests.addCurve(f"{policy.name}-{hpatyp} CReg:{round(env_test.getRegretPerformance(),2)}", y=env_test.resultPerWorkload(), perf=env_test.getRegretPerformance())

env_test.close()

In [ ]:
#hpa_agent.showPredictFig(head_title=f"TEST/{policy.name}", renderer=PLOTLY_RENDERER)

### Baseline test with Basic policy 

In [ ]:
BASIC_OBSERVATION_SPACE_ELEMS = [
    "sysbench_filtered.latency_mean",    # [0, 1000] ms. Filtered latency from client. In inference/live mode, it should be a predicted value (see predictor.ipynb using XGBoost)
    "latency_threshold",                 # Latency threshold for the SLA
    "objective_gap",
    "latency_mean_min",
    "latency_mean_max",
]

from policy_baselines import BasicPolicy

In [ ]:
if LABO_EXPE:
    ds_test.prepareContext(BASIC_OBSERVATION_SPACE_ELEMS)

    env_test = VSMonitor(VSEnv(ds_test, reward=reward_test, notify_react=TEST_USE_SLA_PROTECT, max_steps_per_episode=TEST_STEPS_PER_EPISODE, verbose=1))

    policy = BasicPolicy(k_arms=3, stay_arm=K_ARM_STAY, ctx=ds_test.contextElems(), debug=False)
    basic_agent = VSAgent(policy)
    # No traning with this policy
    # Test it!
    basic_agent.predict(env_test, episodes_max=TEST_EPISODES_COUNT)
    
    graph_tests.addCurve(f"{policy.name} CReg:{round(env_test.getRegretPerformance(),2)}", y=env_test.resultPerWorkload(), perf=env_test.getRegretPerformance())

    env_test.close()


### Baseline test with BufFillingRate Policy

In [ ]:
BFR_OBSERVATION_SPACE_ELEMS = [
    "observation.cache_used_pages_ratio"
]

from policy_baselines import BufFillingRatePolicy 


In [ ]:
ds_test.prepareContext(BFR_OBSERVATION_SPACE_ELEMS)

env_test = VSMonitor(VSEnv(ds_test, reward=reward_test, notify_react=TEST_USE_SLA_PROTECT, max_steps_per_episode=TEST_STEPS_PER_EPISODE, verbose=1))

policy = BufFillingRatePolicy(reward_test.actions, ctx=ds_test.contextElems(), threshold=0.994, debug=False)
bfr_agent = VSAgent(policy)
# No traning with this policy
# Test it!
bfr_agent.predict(env_test, episodes_max=TEST_EPISODES_COUNT)

graph_tests.addCurve(f"{policy.name} CReg:{round(env_test.getRegretPerformance(),2)}", y=env_test.resultPerWorkload(), perf=env_test.getRegretPerformance())


In [ ]:
#bfr_agent.showPredictFig(head_title="TEST/Basic", renderer=PLOTLY_RENDERER)

## Results comparaison

In [ ]:
fig = graph_tests.figure()
fig.show(renderer=None) # svg is cleaner but not in Gitlab. None is the best in VSCode
fig.show(renderer=PLOTLY_RENDERER) # svg is cleaner but not in Gitlab. None is the best in VSCode